In [ ]:
from codesim.problem import StraightLine 

gen = StraightLine(n_ops=20, 
                   n_vars=3,
                   n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()


In [1]:
from codesim.problem import CriticalPath 

gen = CriticalPath(n_ops=20, 
                   n_vars=4,
                   len_critical_path=10)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [2]:
from codesim.problem import ParallelPaths 

gen = ParallelPaths(n_ops=20, 
                    n_vars=3,
                    n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [3]:
from codesim.problem import Loops 

gen = Loops(n_loops=5, 
            n_noisy_loops=2,
            min_loop_length=3,
            max_loop_length=5)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [4]:
from codesim.problem import Sort 

gen = Sort(n_vars=10,
           ascending=True)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [23]:
import sys
from codesim.kim_schuster import real_main
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/kim_schuster.py", 
            "--max_items_per_box", "3",
            "--num_samples", "30"]

real_main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Namespace(object_vocabulary_file='data/objects_with_bnc_frequency.csv', disjoint_object_vocabulary_file=None, disjoint_object_splits='test', output_dir='data/boxes', seed=2255, num_boxes=7, expected_num_items_per_box=1, max_items_per_box=3, num_samples=30, num_operations=10, disjoint_numops=False, zero_shot=False, include_modifiers='never', omit_modifiers_in_ops='never', alternative_forms='never', rarify=False, all_contents_operation=False)
finished sampling 60 sequences of length 10.
Saved outputs to data/boxes.
